In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

lowpoint = pd.read_csv('/Users/mattmastin/Desktop/Battery_Research/lowpoint_data.csv')
midpoint = pd.read_csv('/Users/mattmastin/Desktop/Battery_Research/midpoint_data.csv')
highpoint = pd.read_csv('/Users/mattmastin/Desktop/Battery_Research/highpoint_data.csv')
stretchpoint = pd.read_csv('/Users/mattmastin/Desktop/Battery_Research/stretchpoint_data.csv')

lowpoint.shape, midpoint.shape, highpoint.shape, stretchpoint.shape

((657, 4), (658, 1), (657, 4), (657, 4))

In [15]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [16]:
lowpoint.head()

,Rate,Measure Names,Max Demand Rate ($/kW),Measure Values
0,Time-Of-Use_-_General_Service_-_Large_TOU-8_Op...,kWh Savings ($),$16.23,2419316.75
1,Time-Of-Use_-_General_Service_-_Large_TOU-8_Op...,kWh Savings ($),$38.02,270269.09
2,Time-Of-Use_-_General_Service_-_Large_TOU-8_Op...,kWh Savings ($),$15.57,2977387.00
3,Time-of-use_-_General_Service_-_Demand_Metered...,kWh Savings ($),$36.38,426092.75
4,Time-of-use_-_General_Service_-_Demand_Metered...,kWh Savings ($),$11.05,742679.00


In [17]:
midpoint.head()

,,,midpoint_data
Rate,Measure Names,Max Demand Rate ($/kW),Measure Values
Time-Of-Use_-_General_Service_-_Large_TOU-8_Option_R_(under_2kV),kWh Savings ($),$16.23,2072950
Time-Of-Use_-_General_Service_-_Large_TOU-8_Option_B_(under_2_kV),kWh Savings ($),$38.02,225633.075
Time-Of-Use_-_General_Service_-_Large_TOU-8_Option_A_(Under_2_kV),kWh Savings ($),$15.57,2611630
Time-of-use_-_General_Service_-_Demand_Metered-Super_Off_Peak_TOU-GS-3-SOP_(2kv-50kv),kWh Savings ($),$36.38,359686.75


In [18]:
highpoint.head()

,Rate,Measure Names,Max Demand Rate ($/kW),Measure Values
0,Time-Of-Use_-_General_Service_-_Large_TOU-8_Op...,kWh Savings ($),$16.23,1898552.50
1,Time-Of-Use_-_General_Service_-_Large_TOU-8_Op...,kWh Savings ($),$38.02,191859.98
2,Time-Of-Use_-_General_Service_-_Large_TOU-8_Op...,kWh Savings ($),$15.57,2359980.00
3,Time-of-use_-_General_Service_-_Demand_Metered...,kWh Savings ($),$36.38,254879.50
4,Time-of-use_-_General_Service_-_Demand_Metered...,kWh Savings ($),$11.05,591887.50


In [19]:
stretchpoint.head()

,Rate,Measure Names,Max Demand Rate ($/kW),Measure Values
0,Time-Of-Use_-_General_Service_-_Large_TOU-8_Op...,kWh Savings ($),$16.23,2680804.50
1,Time-Of-Use_-_General_Service_-_Large_TOU-8_Op...,kWh Savings ($),$38.02,323310.12
2,Time-Of-Use_-_General_Service_-_Large_TOU-8_Op...,kWh Savings ($),$15.57,3333423.50
3,Time-of-use_-_General_Service_-_Demand_Metered...,kWh Savings ($),$36.38,505054.75
4,Time-of-use_-_General_Service_-_Demand_Metered...,kWh Savings ($),$11.05,833298.25
